In [2]:
import pandas as pd
import numpy as np


In [3]:
varE = "Monto Plazo Fijo"

df = pd.DataFrame(data={varE: ["0-15000","15000-30000","30000-45000","45000-60000"], 'f':[50,65,45,20]})

df[["Li","Ls"]] = df[varE].str.split("-",expand=True).astype("int") 
df["Pm"] = (df["Ls"] + df["Li"])  / 2
df["F"] = df["f"].cumsum()


print(df)

  Monto Plazo Fijo   f     Li     Ls       Pm    F
0          0-15000  50      0  15000   7500.0   50
1      15000-30000  65  15000  30000  22500.0  115
2      30000-45000  45  30000  45000  37500.0  160
3      45000-60000  20  45000  60000  52500.0  180


In [10]:
promedio = round((df["f"] * df["Pm"]).sum() / df["f"].sum(),4)

varianza = (df["f"] * (df["Pm"] - promedio)**2).sum() / (df["f"].sum() - 1)
desviacion = varianza ** 0.5


print(f"El promedio es ${round(promedio,4)} con un coeficiente de variacion de {round((desviacion / promedio) * 100,4)}% por lo tanto no son homogeneos")

El promedio es $25416.6667 con un coeficiente de variacion de 57.2074% por lo tanto no son homogeneos


In [12]:
total_frecuencias = df['f'].sum()
intervalo_percentil = df[df['F'] >= total_frecuencias * 0.8].iloc[0]
ancho_intervalo = intervalo_percentil["Ls"] - intervalo_percentil["Li"]

percentilochenta = intervalo_percentil["Li"] + (0.8 * total_frecuencias - intervalo_percentil["F"] + intervalo_percentil['f']) / intervalo_percentil["f"] * ancho_intervalo

print(f"El cliente debera tener minimo ${round(percentilochenta,4)}")

El cliente debera tener minimo $39666.6667


In [13]:
anterior = df.loc[df['f'].idxmax()-1]
actual = df.loc[df['f'].idxmax()]
siguiente = df.loc[df['f'].idxmax()+1]

ancho_intervalo = actual["Ls"] - actual["Li"]

moda = actual["Li"] + ancho_intervalo * (actual["f"] - anterior["f"]) / ((actual["f"] - anterior["f"]) + (actual["f"] - siguiente["f"]))


print(f"El monto mas frecuente es ${round(moda,4)} por lo que no se lanzara la campaña.")


El monto mas frecuente es $21428.5714 por lo que no se lanzara la campaña.


In [7]:
def calcularInverso(valor, df: pd.DataFrame):
    # Buscar el intervalo
    ia = None
    ie = None
    for index, row in df.iterrows():
        if row['Li'] <= valor <= row['Ls']:
            ie = row
            if index != 0:
              ia = df.loc[index-1]['F']
            elif index == 0:
                ia = 0
            break
    final = (valor - ie['Li']) / \
        (ie['Ls'] - ie['Li'])
    final = final * ie['f']
    final = final + ia
    final = final * 100
    final = final / df['F'].max()
    return final

In [15]:
percenti = calcularInverso(11000,df)
print(f"El porcentaje de clientes que invierte como minimo $11000 es el {round(percenti,4)}%")

El porcentaje de clientes que invierte como minimo $11000 es el 20.3704%


In [18]:
asimetria = "asimetria positiva por lo que podemos decir que la mayoria de las inversiones son mayores al promedio" if promedio > moda else "asimetria negativa por lo que podemos decir que la mayoria de las inversiones son menores al promedio"

print("El rendimiento posee una", asimetria)

El rendimiento posee una asimetria positiva por lo que podemos decir que la mayoria de las inversiones son mayores al promedio
